In [10]:
from backend.trips import TripPlanner
from backend.connect_to_api import ResRobot
import pandas as pd
import requests
import folium

resrobot = ResRobot()
origin_id = input("Enter your location: ")
destination_id = input("Enter your destination: ")    

In [11]:
from datetime import datetime

class DepaturePlanner(TripPlanner):

planner = DepaturePlanner(origin_id, destination_id)
print(planner.next_available_trips_today()) # Returns a list of DataFrame objects DepaturePlanner(TripPlanner):
    
    def __init__(self, origin_id, destination_id):
        super().__init__(origin_id, destination_id)
        self.origin_id = origin_id
        self.destination_id = destination_id
        self.access_id_from_location(self.origin_id)
        self.access_id_from_location(self.destination_id)
        self.trips = resrobot.trips(self.origin_id, self.destination_id)
        
    def access_id_from_location(self, location):
        url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={self.API_KEY}"
        response = requests.get(url)
        result = response.json()
                            
        if result.get("StopLocation", None):
            return result["StopLocation"][0]["id"]
        else:
            return None
        
    def timetable_departure(self, location_id):
        url = f"https://api.resrobot.se/v2.1/departureBoard?id={location_id}&format=json&accessId={self.API_KEY}"
        response = requests.get(url)
        result = response.json()
        return result

    def next_available_trips_today(self) -> list[pd.DataFrame]:
        """Fetches all available trips today between the origin_id and destination_id
        It returns a list of DataFrame objects, where each item corresponds to a trip
        """
        today = datetime.now().date()
        trips_today = []

        trips = resrobot.trips(self.origin_id, self.destination_id)
        if trips is None:
            return trips_today

        for trip in trips:
            if isinstance(trip, dict):
                leglist = trip.get("LegList", {}).get("Leg", [])
                df_legs = pd.DataFrame(leglist)
                df_stops = pd.json_normalize(df_legs["Stops"].dropna(), "Stop", errors="ignore")
                df_stops["time"] = df_stops["arrTime"].fillna(df_stops["depTime"])
                df_stops["date"] = df_stops["arrDate"].fillna(df_stops["depDate"])

                if df_stops["date"].iloc[0] == today.strftime("%Y-%m-%d"):
                    trips_today.append(
                        df_stops[
                            [
                                "name",
                                "extId",
                                "lon",
                                "lat",
                                "depTime",
                                "depDate",
                                "arrTime",
                                "arrDate",
                                "time",
                                "date",
                            ]
                        ]
                    )

        return trips_today

planner = DepaturePlanner(origin_id, destination_id)
print(planner.next_available_trips_today()) # Returns a list of DataFrame objects

IndentationError: unexpected indent (774511691.py, line 71)

In [5]:
class DepaturePlanner(TripPlanner):
    
    def __init__(self, origin_id, destination_id):
        super().__init__(origin_id, destination_id)
        self.origin_id = origin_id
        self.destination_id = destination_id
        self.access_id_from_location(self.origin_id)
        self.access_id_from_location(self.destination_id)
        self.trips = resrobot.trips(self.origin_id, self.destination)
        
    def access_id_from_location(self, location):
        url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={self.API_KEY}"
        response = requests.get(url)
        result = response.json()
                            
       
    def timetable_departure(self, location_id):
        url = f"https://api.resrobot.se/v2.1/departureBoard?id={location_id}&format=json&accessId={self.API_KEY}"
        location_id = origin_id
        response = requests.get(url)
        result = response.json()
        return result


    def timetable_arrival(self, location_id):
        url = f"https://api.resrobot.se/v2.1/arrivalBoard?id={location_id}&format=json&accessId={self.API_KEY}"
        location_id = destination_id
        response = requests.get(url)
        result = response.json()
        return result
    
    
print(destination_id)

malmö
